In [2]:
import cv2 as cv
import time
import numpy as np
from time import sleep
import RPi.GPIO as GPIO
import math 

capture_object = cv.VideoCapture(0)

n = 1

GPIO.setmode(GPIO.BOARD)

IN1 = 15
IN2 = 16
ENA = 29
IN3 = 18
IN4 = 22
ENB = 31

GPIO.setup(ENA, GPIO.OUT)
GPIO.setup(IN1, GPIO.OUT)
GPIO.setup(IN2, GPIO.OUT)
GPIO.setup(IN3, GPIO.OUT)
GPIO.setup(IN4, GPIO.OUT)
GPIO.setup(ENB, GPIO.OUT)

ena_pin_right = GPIO.PWM(ENA, 1000)
ena_pin_right.start(0)

ena_pin_left = GPIO.PWM(ENB, 1000)
ena_pin_left.start(0)

GPIO.output(IN1, GPIO.LOW)
GPIO.output(IN2, GPIO.HIGH)
GPIO.output(IN3, GPIO.LOW)
GPIO.output(IN4, GPIO.HIGH)

ena_pin_right.ChangeDutyCycle(50)
ena_pin_left.ChangeDutyCycle(50)

while 1:

    ret, img = capture_object.read()
    hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    
    #GREEN
    lower = np.array([60, 100, 50])
    upper = np.array([80, 255, 255])
    
    #Yellow
    #lower = np.array([15, 93, 0])
    #upper = np.array([50, 255, 255])
    
    mask = cv.inRange(hsv, lower, upper)
    
    M = cv.moments(mask)
    x = 320
    y = 0
    print(M["m00"] / 255)
    
    if M["m00"] != 0:
        x = int(M["m10"] / M["m00"])
        y = int(M["m01"] / M["m00"])
        
    err_area = - ((M["m00"]) / 255) + 20000
    w = 640
    center_x = w / 2
    err_angle = center_x - x
    #FOR STRAIGHT FACING CAMERA TRACKING
    k_area = 0.005
    k_angle = 0.075
    p_area = err_area * k_area
    p_angle = err_angle * k_angle
    right_M = p_area + p_angle
    left_M = p_area - p_angle
    #FOR FACING RIGHT CAMERA TRACKING (Camera is turned right)
    k_area = 0.003
    k_angle = 0.15
    p_area = err_area * k_area
    p_angle = err_angle * k_angle
    left_M = p_area - p_angle
    right_M = p_area + p_angle
    #FOR FACING LEFT CAMERA TRACKING (Camera is turned left)
    k_area = 0.003
    k_angle = 0.15
    p_area = err_area * k_area
    p_angle = err_angle * k_angle
    right_M = p_area + p_angle
    left_M = p_area - p_angle
    
    if right_M > 80:
        right_M = 80
    if right_M < 0:
        right_M = 0
    if left_M > 80:
        left_M = 80
    if left_M < 0:
        left_M = 0
    if (M["m00"] / 255) < 1000:
        right_M = 0
        left_M = 0
    ena_pin_right.ChangeDutyCycle(left_M)
    ena_pin_left.ChangeDutyCycle(right_M)
    print(left_M)
    print(right_M)

ena_pin_right.stop()
ena_pin_left.stop()

print(n)

ModuleNotFoundError: No module named 'cv2'